In [3]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import Window

In [4]:
# Initialize Spark
spark = SparkSession.builder \
    .appName("EcommerceAnalysis") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .config("spark.local.dir", "I:/spark_temp") \
    .getOrCreate()

In [5]:
# Load data
df = spark.read.csv('../data/raw/2019-Oct.csv', header=True, inferSchema=True)

In [8]:
df.select('brand').distinct().count()

3446

## Basic Data Overview

In [6]:
print(f"Number of records in 2019-oct: {df.count():,}")

Number of records in 2019-oct: 42,448,764


In [7]:
# Schema of our data:
df.printSchema()

root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)



In [8]:
# Get basic statistics about data
df.describe().show()

+-------+----------+--------------------+--------------------+-------------------+--------+-----------------+-------------------+--------------------+
|summary|event_type|          product_id|         category_id|      category_code|   brand|            price|            user_id|        user_session|
+-------+----------+--------------------+--------------------+-------------------+--------+-----------------+-------------------+--------------------+
|  count|  42448764|            42448764|            42448764|           28933155|36335756|         42448764|           42448764|            42448762|
|   mean|      NULL|1.0549932375842676E7|2.057404237936260...|               NULL|     NaN|290.3236606848809|5.335371475081686E8|                NULL|
| stddev|      NULL|1.1881906970608136E7|1.843926466140411...|               NULL|     NaN|358.2691553394021|1.852373817465431E7|                NULL|
|    min|      cart|             1000978| 2053013552226107603|    accessories.bag|  a-case|   

In [9]:
df.limit(100).sort('user_session').show(30)

+-------------------+----------+----------+-------------------+--------------------+------------+-------+---------+--------------------+
|         event_time|event_type|product_id|        category_id|       category_code|       brand|  price|  user_id|        user_session|
+-------------------+----------+----------+-------------------+--------------------+------------+-------+---------+--------------------+
|2019-10-01 02:00:50|      view|   1005105|2053013555631882655|electronics.smart...|       apple|1415.48|529755884|0b828fb6-99bd-4d2...|
|2019-10-01 02:00:05|      view|   1480613|2053013561092866779|   computers.desktop|      pulser| 908.62|512742880|0d0d91c2-c9c2-4e8...|
|2019-10-01 02:00:22|      view|   1480714|2053013561092866779|   computers.desktop|      pulser| 921.49|512742880|0d0d91c2-c9c2-4e8...|
|2019-10-01 02:00:30|      view|   3701388|2053013565983425517|appliances.enviro...|    dauscher|  33.21|515342595|0e30e1c0-4d3e-4e1...|
|2019-10-01 02:01:00|      view|   370106

As we can see, we have data that can be defined as duplicative. These are rows in which all columns are repeated except the 'event_time' column. These could be legitimate repeated views if the user refreshed the page or might be system-generated events (e.g., auto-refresh). So we have to get rig of them.

In [10]:
# Number of unique users, products and categories:
print(f"Unique users: {df.select('user_id').distinct().count():,}")
print(f"Unique products: {df.select('product_id').distinct().count():,}")
print(f"Unique categories: {df.select('category_id').distinct().count():,}")

Unique users: 3,022,290
Unique products: 166,794
Unique categories: 624


In [11]:
df_small = df.limit(200)

In [12]:
df_cleaned = df.dropDuplicates(['event_type', 'product_id', 'category_id', 'category_code', 'brand', 'price' , 'user_id', 'user_session'])



In [13]:
df_cleaned.sort('user_session').show(100)

+-------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+
|         event_time|event_type|product_id|        category_id|       category_code|   brand|  price|  user_id|        user_session|
+-------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+
|2019-10-06 16:26:10|      cart|   1801723|2053013554415534427|electronics.video.tv|     tcl| 135.65|557388939|                NULL|
|2019-10-25 12:36:14|      cart|   1004767|2053013555631882655|electronics.smart...| samsung| 246.52|549825742|                NULL|
|2019-10-18 12:54:45|      view|  54900011|2146660887203676486|     apparel.costume|    NULL|  64.35|515483062|00000042-3e3f-42f...|
|2019-10-31 07:28:21|      view|   1005104|2053013555631882655|electronics.smart...|   apple| 993.27|513782162|00000056-a206-40d...|
|2019-10-31 07:31:25|      view|   1005212|2053013555631882655|electr

In [14]:
df.count()

42448764

In [15]:
df_cleaned.count()

29138897